# 要約 
このJupyter Notebookでは、「LMSYS - Chatbot Arena」コンペティションのためにユーザーの好みに基づいた予測を行うモデルの構築に取り組んでいます。具体的には、2つの異なる応答からどちらがユーザーに好まれるかを予測し、その結果を確率形式で出力することを目指しています。

### 問題の概要
Notebookは、与えられたチャットボットの応答に対して、どちらが優れているかを判断するための機械学習モデルを開発することに焦点を当てています。ユーザーが与えたプロンプトに対して生成された2つの応答を基に、いずれかの応答が優れているか、または同等であるかを予測します。

### 使用されている手法・ライブラリ
- **Transformersライブラリ**: Hugging FaceのTransformersを利用して大規模言語モデル（特にLlamaモデル）を扱います。
- **Datasetライブラリ**: PyTorchのデータセット、特に`datasets`ライブラリを使用してデータの読み込みと前処理を行います。
- **PEFT (Parameter-Efficient Fine-Tuning)**: モデルのファインチューニングには、データ効率を高めるためにLoRA（Low-Rank Adaptation）を利用しています。
- **Scikit-learn**: おもに評価メトリクス（精度と対数損失の計算）を行うために使用されています。
- **PandasとNumPy**: データ操作に利用され、特にデータフレームを使用してデータの読み込みや変換を行っています。

### 主要なプロセス
1. **データのセットアップ**: Kaggleのデータセットを読み込んで20行分のサンプルデータを取得し、ラベルをエンコードします。
2. **トークナイゼーションと前処理**: 入力データのトークン化を行い、モデルに適した形式に整形します。
3. **モデルアーキテクチャの定義**: カスタムLlamaモデルを定義し、適切なロジックを実装します。損失計算やトークンシフトなどの処理も含まれています。
4. **トレーニングの設定**: トレーニング引数を設定し、評価戦略や最適化手法を明確にします。
5. **モデルのトレーニング**: Trainerを使用してモデルをトレーニングし、トレーニング済みモデルを保存します。
6. **推論プロセスの実装**: 新しいデータに対してモデルを評価し、応答の優劣に基づく確率を算出します。

### 結果の出力
最終的に、Notebookは選択された応答に対するモデルAとモデルBの勝利確率、及び引き分け確率を返す結果データフレームを生成します。この確率は、提出用のCSVファイルのフォーマットに適合しています。

このノートブックは、ユーザーの好みを効果的に予測するためのモデル構築とその実行フローを示しており、実践的な機械学習のスキルを活用しています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に基づいて、機械学習や深層学習の初心者がつまずきそうな専門用語の簡単な解説を示します。特に、実務経験がない場合やこのノートブック特有のドメイン知識に関するものに焦点を当てています。

### 専門用語の解説

1. **LoraConfig**: Low-Rank Adaptation（LoRA）に関連する設定クラス。大規模言語モデル（LLM）のパラメータを効率的に調整するために使用される。学習に必要なパラメータ数を劇的に減らし、特にメモリ制限のある環境でのトレーニングを可能にする。

2. **k-bitトレーニング**: モデルの重みをkビットの精度で保存し、計算にもkビットを使用することで、メモリ効率を高める技術。トレーニングや推論が高速化し、リソースの節約が図られるが、通常の浮動小数点数よりも精度が低くなる。

3. **DataCollator**: トレーニングデータをバッチ化する際に、データを適切な形に整形するクラス。例えば、異なる長さのサンプルを同じ形に整えるためにパディングを行う。

4. **CausalLMOutputWithPast**: 自然言語生成モデルの出力に特化したクラス。過去の隠れ層の状態（hidden states）を保持しつつ、モデルの予測結果や損失を返す。

5. **Attention Mask**: モデルに入力を与える際に、どのトークンがモデルによって処理されるべきかを指定するためのマスク。通常、パディングされた部分を無視させるために使用される。

6. **shift_logits**: モデルが次のトークンを予測するために、以前のトークンの予測結果から1ステップ分シフトさせたロジット。次のトークン予測のためには、前のトークンまでの情報が必要なため。

7. **Logits**: モデルが出力する生の予測値群。通常は、クラスごとのスコアで、これにソフトマックス関数を適用することで確率に変換される。

8. **Metric for best model**: モデルの性能を評価するための指標。特に、トレーニング中に最良のモデルを保存する際に基準となるメトリクスを指定する。ここでは「log_loss」が使われている。

9. **Gradient Accumulation**: バッチサイズを増やさずに、数ステップの勾配を結合してパラメータを更新する手法。メモリを節約しつつ、大きなバッチサイズの効果を得ることができる。

10. **np.isin**: NumPyの関数で、指定した配列内の要素が別の配列に存在するかどうかを判断する。データをフィルタリングする際に便利。

これらの用語は、実務経験がないと具体的な意味や用途がわかりにくいことがありますが、ノートブックの文脈で何を行おうとしているかを理解する助けになります。

---


# セットアップ


In [ ]:
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
import os
import copy
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from datasets import Dataset
from scipy.special import softmax
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BitsAndBytesConfig,
    LlamaPreTrainedModel,
    LlamaModel,
    AutoTokenizer,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
)
from transformers.modeling_outputs import CausalLMOutputWithPast
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

## モデルの設定


In [ ]:
TRAIN_CSV = "/kaggle/input/lmsys-chatbot-arena/train.csv"
model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 1024
target_columns = ['winner_model_a', 'winner_model_b', 'winner_tie']
columns_to_vectorize = ["prompt", "response_a", "response_b"]

## サンプルデータの読み込み


In [ ]:
train = pd.read_csv(TRAIN_CSV)
train = train.head(20)  # データセットの最初の20行を取得
train['label'] = train[target_columns].idxmax(axis=1)  # 各行の最大値のインデックスを 'label' 列に追加
label_encoder = LabelEncoder()
train['label'] = label_encoder.fit_transform(train['label'])  # 'label' 列をエンコード
train = train[columns_to_vectorize + ['label']]  # 必要な列のみを残す

## トークナイザーとデータセットの前処理


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path)  # モデルからトークナイザーを読み込む
tokenizer.add_eos_token = True  # 終端トークンを追加
tokenizer.padding_side = 'right'  # パディングを右側に設定

# 各ラベルに対するトークンIDを取得
LABEL_IDS = [tokenizer(i, add_special_tokens=False)["input_ids"][0] for i in ['a', 'b', 'tie']]

# 各サンプルをトークナイズする関数
def tokenize(example, tokenizer):
    prompt = tokenizer('<prompt>: ' + " ".join(eval(example['prompt'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_a = tokenizer('\n\n<response_a>: ' + " ".join(eval(example['response_a'], {"null": ""})), add_special_tokens=False)["input_ids"]
    response_b = tokenizer('\n\n<response_b>: ' + " ".join(eval(example['response_b'], {"null": ""})), add_special_tokens=False)["input_ids"]
    
    # 最大長を超える場合、プロンプトと応答を切り詰める
    if len(prompt + response_a + response_b) > MAX_LENGTH:
        prompt = tokenizer('<prompt>: ' + eval(example['prompt'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:256]
        response_a = tokenizer('\n\n<response_a>: ' + eval(example['response_a'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
        response_b = tokenizer('\n\n<response_b>: ' + eval(example['response_b'], {"null": ""})[-1], add_special_tokens=False)["input_ids"][:512]
    
    extra_prompt = tokenizer('\n\n---------\nWhich is the better response for the prompt ? a or b or tie ?\n\nAnswer: ', add_special_tokens=False)["input_ids"]

    label_token_id = LABEL_IDS[int(example['label'])]  # ラベルのトークンIDを取得
    input_ids = [tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt + [label_token_id] + [tokenizer.eos_token_id]
    attention_mask = len(input_ids) * [1]  # アテンションマスクの作成
    labels = [-100] * len([tokenizer.bos_token_id] + prompt + response_a + response_b + extra_prompt) + [label_token_id] + [tokenizer.eos_token_id]
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
def load_data(df, tokenizer):
    raw_datasets = Dataset.from_pandas(df)  # Pandasデータフレームをデータセットに変換
    tokenized_datasets = raw_datasets.map(
        tokenize, 
        remove_columns=raw_datasets.column_names,  # 元の列を削除
        fn_kwargs={'tokenizer': tokenizer}
    )
    return tokenized_datasets

n_splits = 5  # 交差検証の分割数
fold_idx = 0  # 使用するフォールドのインデックス
ds = load_data(train, tokenizer)  # トークン化されたデータをロード
folds = [
    (
        [i for i in range(len(ds)) if i % n_splits != fold_idx],  # トレーニングインデックス
        [i for i in range(len(ds)) if i % n_splits == fold_idx]  # 評価インデックス
    ) 
    for fold_idx in range(n_splits)
]
train_idx, eval_idx = folds[fold_idx]  # トレーニングと評価のインデックスを取得

## メトリクス


In [ ]:
def compute_metrics(pred):
    logits, labels = pred  # 予測結果とラベルを取得
    preds = logits.argmax(axis=-1)  # 最も高いロジット値を持つインデックスを取得
    label_tokens_ids = np.array(LABEL_IDS)
    index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}  # インデックスのマッピングを作成
    labels = labels[np.isin(labels, label_tokens_ids)]  # 有効なラベルのみをフィルタリング
    labels = np.array([index_mapping[label.item()] for label in labels])  # マッピングされたラベルを取得
    acc = accuracy_score(labels, preds)  # 精度を計算
    probs = softmax(logits, axis=-1)  # ソフトマックスを計算して確率を取得
    log_loss_ = log_loss(labels, probs)  # ログロスを計算
    return {'accuracy': acc, 'log_loss': log_loss_}  # 精度とログロスを辞書で返す

## モデル


In [ ]:
class CustomLlama3(LlamaPreTrainedModel):
    _tied_weights_keys = ["lm_head.weight"]  # 重みの関連付けを作成
    def __init__(self, config):
        super().__init__(config)
        self.model = LlamaModel(config)  # Llamaモデルの構成
        self.vocab_size = config.vocab_size  # 単語のボキャブラリサイズ
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)  # 言語モデルのヘッド
        self.post_init()  # 初期化後の処理

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        position_ids=None,
        past_key_values=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        cache_position=None,
    ):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            cache_position=cache_position,
        )
        hidden_states = outputs[0]  # 隠れ層の状態を取得
        if self.config.pretraining_tp > 1:  # モデル並列化を使用する場合
            lm_head_slices = self.lm_head.weight.split(self.vocab_size // self.config.pretraining_tp, dim=0)
            logits = [F.linear(hidden_states, lm_head_slices[i]) for i in range(self.config.pretraining_tp)]
            logits = torch.cat(logits, dim=-1)  # ロジットを結合
        else:
            logits = self.lm_head(hidden_states)  # ロジットを算出

        logits = logits.float()  # ロジットを浮動小数点数に変換

        loss = None
        if labels is not None:
            # 次のトークンを予測するためにシフトする
            shift_logits = logits[..., :-1, :].contiguous()  # シフトしたロジット
            shift_labels = labels[..., 1:].contiguous()  # シフトしたラベル
            # トークンをフラット化する
            loss_fct = nn.CrossEntropyLoss()  # クロスエントロピー損失を定義
            shift_logits = shift_logits.view(-1, self.config.vocab_size)  # フラット化
            shift_labels = shift_labels.view(-1)  # フラット化
            # モデル並列化を有効にする
            shift_labels = shift_labels.to(shift_logits.device)

            label_tokens_ids = torch.tensor(LABEL_IDS, device=shift_labels.device)  # ラベルのトークンIDをテンソルに変換
            index_mapping = {value.item(): idx for idx, value in enumerate(label_tokens_ids)}
            true_labels = shift_labels[torch.isin(shift_labels, label_tokens_ids)]  # 有効なラベルのみを取得
            true_labels = torch.tensor([index_mapping[label.item()] for label in true_labels], device=true_labels.device)
            true_logits = shift_logits[torch.isin(shift_labels, label_tokens_ids)][:, label_tokens_ids]  # 有効なロジットを取得
            loss = loss_fct(true_logits, true_labels)  # 損失を計算

        return CausalLMOutputWithPast(
            loss=loss,
            logits=true_logits,
        )

In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj', 'v_proj',], 
)

In [ ]:
model = CustomLlama3.from_pretrained(
    model_path, 
    load_in_8bit=True,
    torch_dtype=torch.float16,
    cache_dir="/kaggle/working/model"
)
model.config.use_cache = False  # キャッシュを使用しない
model = prepare_model_for_kbit_training(model)  # k-bitトレーニングの準備
model = get_peft_model(model, peft_config)  # PEFTモデルの取得
print(model)  # モデルの情報を表示
model.print_trainable_parameters()  # 学習可能なパラメータを表示

### トレーニング引数


In [ ]:
args = TrainingArguments(
    output_dir='output',  # 出力ディレクトリ
    overwrite_output_dir=True,  # 上書きを許可
    evaluation_strategy="epoch",  # エポックごとに評価
    save_strategy="steps",  # ステップごとに保存
    save_steps=5,  # 5ステップごとに保存
    save_total_limit=1,  # 保存される最大数
    logging_strategy="steps",  # ステップごとのロギング
    logging_steps=10,  # 10ステップごとにログを記録
    warmup_steps=20,  # ウォームアップステップ数
    optim="adamw_8bit",  # 最適化手法
    learning_rate=2e-4,  # 学習率
    per_device_train_batch_size=2,  # トレーニングのバッチサイズ（デバイスあたり）
    per_device_eval_batch_size=2,  # 評価のバッチサイズ（デバイスあたり）
    gradient_accumulation_steps=2,  # 勾配の累積ステップ数
    num_train_epochs=1,  # トレーニングエポック数
    fp16=True,  # 混合精度トレーニングを有効にする
    metric_for_best_model="log_loss",  # 最良モデルのメトリクス
    greater_is_better=False,  # メトリクスの良好さ
    report_to="none",
)

# トレーニング（トレーニングまたは推論のいずれかを実行）


In [ ]:
trainer = Trainer(
    args=args,
    model=model,
    train_dataset=ds.select(train_idx),  # トレーニングデータセットを選択
    eval_dataset=ds.select(eval_idx),  # 評価データセットを選択
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),  # データコレータの設定
    compute_metrics=compute_metrics,  # メトリクスの計算
)
trainer.train()  # トレーニングを実行

In [ ]:
model.save_pretrained('pretrained_model')  # モデルを保存

# 推論（トレーニングまたは推論のいずれかを実行）


In [ ]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM,
    target_modules=['q_proj', 'k_proj', 'v_proj',], 
)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# 事前にトレーニングされたモデルとLoraアダプタのファイルパス
model_path = "unsloth/llama-3-8b-Instruct-bnb-4bit"
lora_adapter_path = "/kaggle/working/pretrained_model"

# 原モデルを読み込む
model_1 = AutoModelForCausalLM.from_pretrained(model_path)

# 対応するトークナイザーを読み込む
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Loraの設定
lora_config = peft_config

# 必要に応じてk-bitトレーニングの準備
model_1 = prepare_model_for_kbit_training(model_1)

# モデルにLoraアダプタを適用
model_1 = get_peft_model(model_1, lora_config)

# 保存されたLoraアダプタのパラメータを読み込む
model_1.load_adapter(lora_adapter_path, adapter_name="test")

# 完全なモデルが使用できる状態になりました
model_1.eval()  # 評価モードに設定

In [ ]:
from transformers.data.data_collator import pad_without_fast_tokenizer_warning
import pandas as pd
import numpy as np

def softmax(row):
    e_row = np.exp(row - np.max(row))  # ソフトマックス計算のための安定化
    return e_row / e_row.sum()  # ソフトマックスを計算して正規化

In [ ]:
data = ds.to_pandas()[0:10]  # データセットをPandasデータフレームに変換し最初の10行を取得
data["max_len"] = data["input_ids"].apply(len)  # 各入力の長さを計算
display(data[:3])  # 最初の3行を表示
print()

print(tokenizer.decode(data["input_ids"][0]))  # 0番目の入力のデコード結果を表示

In [ ]:
@torch.no_grad()  # 勾配計算を無効にする
@torch.cuda.amp.autocast()  # 自動混合精度を有効にする
def inference(df, model, device, batch_size=2, max_length=1024):
    a_win, b_win, tie = [], [], []  # 各モデルの勝ち確率を格納するリスト

    model.eval()  # モデルを評価モードに設定
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終了インデックスを決定
        tmp = df.iloc[start_idx:end_idx]  # 現在のバッチデータを取得
        input_ids = tmp["input_ids"].to_list()  # 入力IDをリストに変換
        attention_mask = tmp["attention_mask"].to_list()  # アテンションマスクをリストに変換
        labels = tmp["labels"].to_list()  # ラベルをリストに変換
        inputs = pad_without_fast_tokenizer_warning(
            tokenizer,
            {"input_ids": input_ids, "attention_mask": attention_mask},
            padding="longest",  # 最長の長さに合わせてパディング
            pad_to_multiple_of=None,
            return_tensors="pt",  # Pytorchテンソルとして返す
        )
        input_ids = inputs["input_ids"].to(device)  # デバイスに移動
        attention_mask = inputs["attention_mask"].to(device)  # デバイスに移動
        pad_labels = []  # パディングされたラベルを格納するリスト
        for label in labels:
            label = list(label) + [tokenizer.pad_token_id] * (input_ids[0].shape[0] - label.shape[0])  # ラベルをパディング
            pad_labels.append(label)
        labels = torch.tensor(pad_labels).to(device)  # デバイスに移動
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)  # モデルの出力を取得
        proba = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # ロジットにソフトマックスを適用して確率に変換
        a_win.extend(proba[:, 0].tolist())  # モデルAの勝ち確率を追加
        b_win.extend(proba[:, 1].tolist())  # モデルBの勝ち確率を追加
        tie.extend(proba[:, 2].tolist())  # 引き分けの確率を追加
    df['winner_model_a'] = a_win  # データフレームにモデルAの勝ち確率を追加
    df['winner_model_b'] = b_win  # データフレームにモデルBの勝ち確率を追加
    df['winner_tie'] = tie  # データフレームに引き分けの確率を追加
    return df  # 結果のデータフレームを返す

In [ ]:
result_df = inference(data[0:4], model_1, device, batch_size=2, max_length=1024)  # 推論を実行

proba = result_df[["winner_model_a", "winner_model_b", "winner_tie"]].values  # 確率の値を取得される

result_df.loc[:, "winner_model_a"] = proba[:, 0]  # モデルAの勝ち確率をデータフレームに設定
result_df.loc[:, "winner_model_b"] = proba[:, 1]  # モデルBの勝ち確率をデータフレームに設定
result_df.loc[:, "winner_tie"] = proba[:, 2]  # 引き分けの確率をデータフレームに設定

# 確率のリストをフラットにする
result_df['winner_model_a'] = result_df['winner_model_a'].apply(lambda x: x[0])  
result_df['winner_model_b'] = result_df['winner_model_b'].apply(lambda x: x[0])  
result_df['winner_tie'] = result_df['winner_tie'].apply(lambda x: x[0])  

result_df  # 最終結果のデータフレームを表示